In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold,train_test_split,KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Models
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier

import os
import pandas as pd
import numpy as np
import time


import warnings
warnings.filterwarnings("ignore")

In [2]:
sample_submission = pd.read_csv("/kaggle/input/playground-series-s3e18/sample_submission.csv")
test_data = pd.read_csv("/kaggle/input/playground-series-s3e18/test.csv")
train_data = pd.read_csv("/kaggle/input/playground-series-s3e18/train.csv")

In [3]:
train_data = train_data.drop(columns=['EC3', 'EC4', 'EC5', 'EC6'], axis=1)

In [4]:
y  = train_data[['EC1', 'EC2']]
X  = train_data.drop(columns=['EC1', 'EC2'], axis=1)

clf = XGBClassifier()
pipe = Pipeline([
                ('classifier', clf)
               ])

oof_preds = np.zeros(y.shape)
test_preds = np.zeros((test_data.shape[0], y.shape[1]))
oof_losses = []

kf = KFold(n_splits = 5)

In [5]:
data = kf.split(X,y)

In [6]:
for i, (train_index, val_index) in enumerate(data):
    x_train_kf, x_val_kf = X.iloc[train_index], X.iloc[val_index]
    y_train_kf, y_val_kf = y.iloc[train_index], y.iloc[val_index]
    clf.fit(x_train_kf,y_train_kf)
    
    val_preds = clf.predict_proba(x_val_kf)
    val_preds = np.array(val_preds)[:,:]
    oof_preds[val_index] = val_preds
    
    loss = roc_auc_score(np.ravel(y_val_kf), np.ravel(val_preds))
    oof_losses.append(loss)
    preds = clf.predict_proba(test_data)
    preds = np.array(preds)[:,:] 
    test_preds += preds / 5
    
    print(i , "Loss = ",  loss)
    
    
print("Mean Loss: ",np.mean(oof_losses))

0 Loss =  0.6382214039412171
1 Loss =  0.6358259586596453
2 Loss =  0.652389624432421
3 Loss =  0.6362936414183604
4 Loss =  0.6182740612647155
Mean Loss:  0.6362009379432718


In [7]:
submission = sample_submission
submission.iloc[:,1:] = test_preds

In [8]:
submission

,id,EC1,EC2
0,14838,0.362125,0.807602
1,14839,0.851976,0.984866
2,14840,0.848867,0.821028
3,14841,0.579888,0.858874
4,14842,0.886415,0.941610
...,...,...,...
9888,24726,0.707756,0.938853
9889,24727,0.845647,0.935224
9890,24728,0.280345,0.854974
9891,24729,0.266548,0.915021


In [9]:
submission.to_csv("submission.csv",index=False)